#뇌 MRI 이미지에서 뇌종양 검출
본 과제는 2점 만점이며, 정확도(Accuracy)를 평가 지표로 사용합니다.
Baseline보다 높은 정확도 달성: 2.0점
Baseline의 정확도와 동일(소수 5 번째 자리까지)하거나, 낮은 정확도 달성: 1.0점
미제출: 0점
(주의) 0.60784 이하로 기록된 것은 미제출로 간주됩니다.

현재 Baseline은 Keras 라이브러리를 활용해 구성한 합성곱 신경망 모델을 활용해 도출됐습니다.
데이터 세트 분할 시, random_state 값은 42로 지정했습니다.
수업 시간에 다뤘던 합성곱 신경망 모델 구조 설계 및 하이퍼파라미터와 관련된 내용을 복습하고, 과제 4를 해결해 봅시다.

In [ ]:
import numpy as np

data = np.load("train.npz")
x = data["x"]      # (202, H, W)
y = data["y"]      # ['normal', 'tumor', 'normal', ...]

print(np.unique(y))  # ['normal' 'tumor']

# 문자열 -> 숫자 라벨로 변환
label_map = {"normal": 0, "tumor": 1}
y_num = np.array([label_map[v] for v in y], dtype="int32")


['normal' 'tumor']


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# ======================
# 1. train / test 데이터 로드
# ======================
train_data = np.load("train.npz")
x = train_data["x"]    # (202, H, W)
y = train_data["y"]    # ['normal', 'tumor', ...]

test_data = np.load("test.npz")
x_test = test_data["x"]    # (51, H, W)

# ======================
# 2. 레이블 문자열 -> 숫자 (0/1) 인코딩
# ======================
label_map = {"normal": 0, "tumor": 1}
y_num = np.array([label_map[v] for v in y], dtype="int32")

# ======================
# 3. 이미지 전처리 (채널 차원 추가 + 정규화)
# ======================
x = x.reshape(-1, x.shape[1], x.shape[2], 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, x_test.shape[1], x_test.shape[2], 1).astype("float32") / 255.0

# ======================
# 4. train / validation 분할 (random_state=42 필수)
# ======================
x_train, x_val, y_train, y_val = train_test_split(
    x, y_num, test_size=0.2, random_state=42, stratify=y_num
)

# ======================
# 5. CNN 모델 정의
# ======================
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=x_train.shape[1:]),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # 이진 분류 → sigmoid + 1노드
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

es = EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    restore_best_weights=True
)

# ======================
# 6. 학습
# ======================
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=[es],
    verbose=1
)

# ======================
# 7. 검증 정확도 확인
# ======================
val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print("Validation Accuracy:", val_acc)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.6379 - loss: 4.8596 - val_accuracy: 0.6098 - val_loss: 0.9209
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.7608 - loss: 3.5518 - val_accuracy: 0.6098 - val_loss: 0.9690
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.7608 - loss: 1.4939 - val_accuracy: 0.6098 - val_loss: 2.4726
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.8153 - loss: 2.2170 - val_accuracy: 0.6098 - val_loss: 0.6925
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.8642 - loss: 0.5612 - val_accuracy: 0.6098 - val_loss: 1.3999
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.8752 - loss: 0.6836 - val_accuracy: 0.6098 - val_loss: 1.9832
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9154 - loss: 0.4836 - val_accuracy: 0.6098 - val_loss: 2.2555
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8690 - loss: 0.4446 - val_accuracy: 0.6098 - val_loss:

In [ ]:
# ======================
# 8. 전체 train으로 다시 학습 (선택 사항)
#    검증에서 성능 괜찮으면 전체 데이터로 다시 학습해도 됨
# ======================
model_final = model  # 간단히 재사용해도 되고,
# 필요하면 x, y_num 전체로 다시 fit() 해도 됨.

# ======================
# 9. test 예측
# ======================
y_prob = model_final.predict(x_test).flatten()
y_pred = (y_prob > 0.5).astype(int)  # 0/1로 변환

# ======================
# 10. submission.csv에 기록
# ======================
df = pd.read_csv("submission.csv")
df.dropna(axis=1, inplace=True)

df["result"] = y_pred
df.to_csv("new_submission.csv", index=False)

print("✅ new_submission.csv 저장 완료")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
✅ new_submission.csv 저장 완료


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("submission.csv")

df.dropna(axis=1, inplace=True)

# y_pred: 0/1 예측값
idx_to_label = {0: "normal", 1: "tumor"}
y_pred_label = np.array([idx_to_label[int(v)] for v in y_pred])

df["result"] = y_pred_label

df.to_csv("new_submission.csv", index=False)
print("✅ new_submission.csv 저장 완료")


✅ new_submission.csv 저장 완료


#2트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# 1. 데이터 로드
data = np.load("train.npz")
x = data["x"]       # (202, H, W)
y = data["y"]       # ['normal', 'tumor', ...]

# 레이블 문자열 -> 숫자
label_map = {"normal": 0, "tumor": 1}
y_num = np.array([label_map[v] for v in y], dtype="int32")

# 이미지 전처리: (N, H, W, 1) + [0,1] 정규화
x = x.reshape(-1, x.shape[1], x.shape[2], 1).astype("float32") / 255.0

# train/valid 분할 (random_state=42)
x_train, x_val, y_train, y_val = train_test_split(
    x, y_num, test_size=0.2, random_state=42, stratify=y_num
)


In [ ]:
input_shape = x_train.shape[1:]

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

model = models.Sequential([
    layers.Input(shape=input_shape),
    data_augmentation,

    layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

es = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=[es],
    verbose=1
)


Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.6572 - loss: 9.9127 - val_accuracy: 0.6098 - val_loss: 0.8162
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.5625 - loss: 5.3723 - val_accuracy: 0.6098 - val_loss: 2.8252
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.7030 - loss: 1.3366 - val_accuracy: 0.6098 - val_loss: 2.7768
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5662 - loss: 1.8190 - val_accuracy: 0.6098 - val_loss: 1.5333
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.6760 - loss: 1.3203 - val_accuracy: 0.6098 - val_loss: 2.0052
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.7447 - loss: 0.8031 - val_accuracy: 0.6098 - val_loss: 1.4344
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.7179 - loss: 0.5920 - val_accuracy: 0.6098 - val_loss: 0.7656
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6615 - loss: 1.0753 - val_accuracy: 0.6098 - val_loss:

In [ ]:
y_val_prob = model.predict(x_val).ravel()

best_acc = 0
best_th = 0.5

for th in np.linspace(0.1, 0.9, 81):  # 0.1 ~ 0.9 사이를 0.01 간격으로
    y_val_pred = (y_val_prob >= th).astype(int)
    acc = accuracy_score(y_val, y_val_pred)
    if acc > best_acc:
        best_acc = acc
        best_th = th

print("Best threshold:", best_th)
print("Best val accuracy:", best_acc)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 324ms/step
Best threshold: 0.1
Best val accuracy: 0.6097560975609756


In [ ]:
# 1) 전체 데이터로 다시 학습
data = np.load("train.npz")
x_all = data["x"]
y_all = data["y"]

y_all_num = np.array([label_map[v] for v in y_all], dtype="int32")
x_all = x_all.reshape(-1, x_all.shape[1], x_all.shape[2], 1).astype("float32") / 255.0

model_final = tf.keras.models.clone_model(model)
model_final.build(input_shape=(None,) + input_shape)
model_final.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model_final.fit(
    x_all, y_all_num,
    epochs=best_epochs if 'best_epochs' in globals() else 30,
    batch_size=16,
    verbose=1
)

# 2) test.npz 예측
test_data = np.load("test.npz")
x_test = test_data["x"]
x_test = x_test.reshape(-1, x_test.shape[1], x_test.shape[2], 1).astype("float32") / 255.0

y_test_prob = model_final.predict(x_test).ravel()
y_test_pred_num = (y_test_prob >= best_th).astype(int)

# 숫자 -> 레이블 문자열
idx_to_label = {0: "normal", 1: "tumor"}
y_test_label = np.array([idx_to_label[int(v)] for v in y_test_pred_num])

# 3) submission 저장
df = pd.read_csv("submission.csv")
df.dropna(axis=1, inplace=True)

df["result"] = y_test_label
df.to_csv("new_submission.csv", index=False)
print("✅ new_submission.csv 저장 완료")


Epoch 1/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.5579 - loss: 10.1297
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.7175 - loss: 2.0854
Epoch 3/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6118 - loss: 1.3749
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.6784 - loss: 0.7463
Epoch 5/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.7039 - loss: 0.8984
Epoch 6/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.7079 - loss: 0.6627
Epoch 7/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.6283 - loss: 0.6501
Epoch 8/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.6789 - loss: 0.7960
Epoch 9/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6282 - loss: 0.6384
Epoch 10/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.6590 - loss: 0.7176
Epoch 11/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6252 - loss: 0.7555
Epoch 12/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.6909 - l

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 866ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
✅ new_submission.csv 저장 완료


#3트

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ============================================================
# 1. 데이터 로드
# ============================================================
print("1. 데이터 로드 중...")
try:
    train_data = np.load("train.npz")
    x_train_full = train_data["x"]
    y_train_full = train_data["y"]  # 문자열 라벨 가능

    test_data = np.load("test.npz")
    x_test = test_data["x"]

    df_submission = pd.read_csv("submission.csv")

    # 제출 컬럼 자동 선택
    if "result" in df_submission.columns:
        result_col = "result"
    else:
        result_col = df_submission.columns[1]  # 두 번째 컬럼
    print(f"예측 결과는 '{result_col}' 컬럼에 저장합니다.")

except Exception as e:
    print("파일 오류:", e)
    raise SystemExit()

print(f"전체 학습 샘플 수: {x_train_full.shape[0]}, 이미지 해상도: {x_train_full.shape[1:]}")


# ============================================================
# 2. 데이터 전처리
# ============================================================
print("\n2. 데이터 전처리 및 분할 중...")

# A. 정규화
x_train_full = x_train_full.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# B. 흑백 이미지 채널 추가
if len(x_train_full.shape) == 3:  # (N,H,W)
    x_train_full = np.expand_dims(x_train_full, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)

input_shape = x_train_full.shape[1:]
print("CNN 입력 shape:", input_shape)


# C. 문자열 레이블 처리 (핵심)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train_full)  # 문자열 → 0/1 숫자 변환

print("라벨 인코딩 확인:", list(zip(y_train_full[:5], y_encoded[:5])))


# D. train/validation 분할
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

print(f"train 개수: {x_train.shape[0]}, val 개수: {x_val.shape[0]}")


# ============================================================
# 3. 데이터 증강
# ============================================================
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest",
)

train_generator = datagen.flow(x_train, y_train, batch_size=16)


# ============================================================
# 4. CNN 모델 정의
# ============================================================
print("\n4. CNN 모델 설계 중...")

def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.3),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # 이진 분류
    ])
    return model

model = create_cnn_model(input_shape)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

model.summary()


# ============================================================
# 5. 모델 학습
# ============================================================
print("\n5. 모델 학습 시작...")

early_stopping = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.0001)

history = model.fit(
    train_generator,
    steps_per_epoch=len(x_train) // 16,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print(f"\n최종 검증 정확도: {val_acc:.4f}")


# ============================================================
# 6. 예측 + 제출 파일 생성
# ============================================================
print("\n6. 제출 파일 생성 중...")

# 예측 확률
y_pred_proba = model.predict(x_test)

# 0/1 변환
y_pred = np.round(y_pred_proba).astype(int).flatten()

df_submission[result_col] = y_pred

submission_filename = "my_submission_brain_tumor_xai_04.csv"
df_submission.to_csv(submission_filename, index=False)

print(f"\n✅ 제출 파일 저장 완료: {submission_filename}")
print("이 파일을 스마트클래스에 제출하면 됩니다.")


1. 데이터 로드 중...
예측 결과는 'result' 컬럼에 저장합니다.
전체 학습 샘플 수: 202, 이미지 해상도: (200, 200)

2. 데이터 전처리 및 분할 중...
CNN 입력 shape: (200, 200, 1)
라벨 인코딩 확인: [(np.str_('normal'), np.int64(0)), (np.str_('tumor'), np.int64(1)), (np.str_('normal'), np.int64(0)), (np.str_('tumor'), np.int64(1)), (np.str_('normal'), np.int64(0))]
train 개수: 161, val 개수: 41

4. CNN 모델 설계 중...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 198, 198, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 67712)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    17,334,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,427,457 (66.48 MB)

...


5. 모델 학습 시작...
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 932ms/step - accuracy: 0.5689 - loss: 1.4690 - val_accuracy: 0.5610 - val_loss: 0.6904 - learning_rate: 0.0010
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6250 - loss: 0.6625 - val_accuracy: 0.7561 - val_loss: 0.6864 - learning_rate: 0.0010
Epoch 3/100


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.6391 - loss: 0.6488 - val_accuracy: 0.6829 - val_loss: 0.6517 - learning_rate: 0.0010
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5625 - loss: 0.6130 - val_accuracy: 0.7317 - val_loss: 0.6665 - learning_rate: 0.0010
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.6424 - loss: 0.6581 - val_accuracy: 0.5366 - val_loss: 0.6891 - learning_rate: 0.0010
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8125 - loss: 0.6793 - val_accuracy: 0.7073 - val_loss: 0.6877 - learning_rate: 0.0010
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.6741 - loss: 0.6674 - val_accuracy: 0.6098 - val_loss: 0.6521 - learning_rate: 0.0010
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6875 - loss: 0.6013 - val_accuracy: 0.6098 - val_loss: 0.6492 - learning_rate: 0.0010
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.6301 - loss: 0.6130 - val_accuracy: 

In [5]:
# 숫자(0/1) → 문자열 라벨로 변환
y_pred_labels = label_encoder.inverse_transform(y_pred)

# submission.csv에 기록
df_submission[result_col] = y_pred_labels

# 새로운 제출 파일 이름
submission_filename_label = "my_submission_brain_tumor_xai_04_label.csv"
df_submission.to_csv(submission_filename_label, index=False)

print(f"\n🎉 문자열 라벨 포함 제출 파일 생성 완료: {submission_filename_label}")


🎉 문자열 라벨 포함 제출 파일 생성 완료: my_submission_brain_tumor_xai_04_label.csv


#4트

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout,
    BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


# ==============================================================
# 1. 데이터 로드
# ==============================================================
print("1. 데이터 로드 중...")

train_data = np.load("train.npz")
x_train_full = train_data["x"]
y_train_full = train_data["y"]

test_data = np.load("test.npz")
x_test = test_data["x"]

df_submission = pd.read_csv("submission.csv")

# 제출 컬럼 자동 탐지
result_col = "result" if "result" in df_submission.columns else df_submission.columns[1]

print("데이터 로드 완료!")
print(f"train 샘플: {x_train_full.shape[0]}, test 샘플: {x_test.shape[0]}")
print(f"이미지 shape: {x_train_full.shape[1:]}")


# ==============================================================
# 2. 전처리
# ==============================================================
print("\n2. 데이터 전처리 중...")

# 정규화
x_train_full = x_train_full.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# 흑백이면 채널 추가
if len(x_train_full.shape) == 3:  # (N,H,W)
    x_train_full = np.expand_dims(x_train_full, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)

input_shape = x_train_full.shape[1:]
print("CNN 입력 shape:", input_shape)

# 문자열 라벨 처리
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train_full)

print("라벨 인코딩 예시:", list(zip(y_train_full[:5], y_encoded[:5])))

# Train/Validation split
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

print(f"Train: {x_train.shape[0]}, Validation: {x_val.shape[0]}")


# ==============================================================
# 3. 데이터 증강
# ==============================================================
print("\n3. 데이터 증강 설정...")

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    fill_mode="nearest"
)

train_generator = datagen.flow(x_train, y_train, batch_size=16)


# ==============================================================
# 4. CNN 모델 정의 (고성능 구조)
# ==============================================================
print("\n4. CNN 모델 생성...")

def create_cnn(input_shape):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        GlobalAveragePooling2D(),

        Dense(64, activation='relu'),
        Dropout(0.3),

        Dense(1, activation='sigmoid')  # 이진 분류
    ])
    return model

model = create_cnn(input_shape)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


# ==============================================================
# 5. 모델 학습
# ==============================================================
print("\n5. 모델 학습 시작...")

early_stopping = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.0001)

history = model.fit(
    train_generator,
    steps_per_epoch=len(x_train) // 16,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print(f"\n최종 검증 정확도: {val_acc:.4f}")


# ==============================================================
# 6. 제출 파일 생성
# ==============================================================
print("\n6. 제출 파일 생성 중...")

y_pred_proba = model.predict(x_test)
y_pred = np.round(y_pred_proba).astype(int).flatten()

df_submission[result_col] = y_pred

output_file = "my_submission_brain_tumor.csv"
df_submission.to_csv(output_file, index=False)

print(f"\n🎉 제출 파일 생성 완료: {output_file}")


1. 데이터 로드 중...
데이터 로드 완료!
train 샘플: 202, test 샘플: 51
이미지 shape: (200, 200)

2. 데이터 전처리 중...
CNN 입력 shape: (200, 200, 1)
라벨 인코딩 예시: [(np.str_('normal'), np.int64(0)), (np.str_('tumor'), np.int64(1)), (np.str_('normal'), np.int64(0)), (np.str_('tumor'), np.int64(1)), (np.str_('normal'), np.int64(0))]
Train: 161, Validation: 41

3. 데이터 증강 설정...

4. CNN 모델 생성...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 200, 200, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 200, 200, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 100, 100, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 100, 100, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 100, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 50, 50, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 50, 50, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,889 (398.00 KB)

...


5. 모델 학습 시작...
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 610ms/step - accuracy: 0.5910 - loss: 0.7258 - val_accuracy: 0.6098 - val_loss: 0.6749 - learning_rate: 0.0010
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6875 - loss: 0.7912 - val_accuracy: 0.6098 - val_loss: 0.6734 - learning_rate: 0.0010
Epoch 3/100
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7500 - loss: 0.5753

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.6834 - loss: 0.6193 - val_accuracy: 0.6098 - val_loss: 0.6699 - learning_rate: 0.0010
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7500 - loss: 0.5599 - val_accuracy: 0.6098 - val_loss: 0.6706 - learning_rate: 0.0010
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.7513 - loss: 0.4986 - val_accuracy: 0.6098 - val_loss: 0.6775 - learning_rate: 0.0010
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6250 - loss: 0.6927 - val_accuracy: 0.6098 - val_loss: 0.6783 - learning_rate: 0.0010
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.7785 - loss: 0.4811 - val_accuracy: 0.6098 - val_loss: 0.6773 - learning_rate: 0.0010
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7500 - loss: 0.4584 - val_accuracy: 0.6098 - val_loss: 0.6769 - learning_rate: 0.0010
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.7506 - loss: 0.5005 - val_accuracy:

In [3]:
# ==============================================================
# 7. 숫자 라벨을 원래 문자열 라벨로 변환하여 제출 파일 다시 저장
# ==============================================================

# 0/1 → 문자열 라벨로 되돌리기
y_pred_labels = label_encoder.inverse_transform(y_pred)

# 원래 submission 형식에 문자열 라벨 기록
df_submission[result_col] = y_pred_labels

# 문자열 라벨 버전 새 파일로 저장
output_file_labels = "my_submission_brain_tumor_label.csv"
df_submission.to_csv(output_file_labels, index=False)

print(f"\n🎉 문자열 라벨 포함 제출 파일 생성 완료: {output_file_labels}")


🎉 문자열 라벨 포함 제출 파일 생성 완료: my_submission_brain_tumor_label.csv


# 5트

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2 # 강력한 사전 학습 모델 사용
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ============================================================
# 1. 데이터 로드 및 전처리
# ============================================================
IMG_SIZE = 128 # MobileNetV2는 224x224가 일반적이나, 적은 데이터셋을 위해 더 작게 설정
print("1. 데이터 로드 및 전처리 중...")

try:
    train_data = np.load("train.npz")
    x_train_full = train_data["x"]
    y_train_full = train_data["y"]
    test_data = np.load("test.npz")
    x_test = test_data["x"]
    df_submission = pd.read_csv("submission.csv")
    result_col = "result" if "result" in df_submission.columns else df_submission.columns[1]
except Exception as e:
    print("파일 오류:", e)
    raise SystemExit()

# A. 흑백 채널 -> 3채널(RGB)로 복제 (전이 학습 모델의 필수 조건)
def convert_to_3_channels(data):
    data = data.astype("float32")
    if len(data.shape) == 3: # (N, H, W)
        data = np.expand_dims(data, axis=-1)
    if data.shape[-1] == 1: # (N, H, W, 1)
        data = np.repeat(data, 3, axis=-1) # 3채널 복제
    return data

x_train_full = convert_to_3_channels(x_train_full)
x_test = convert_to_3_channels(x_test)


# B. 리사이즈 (MobileNetV2 입력 크기에 맞춤)
x_train_full = tf.image.resize(x_train_full, (IMG_SIZE, IMG_SIZE)).numpy()
x_test = tf.image.resize(x_test, (IMG_SIZE, IMG_SIZE)).numpy()


# C. 라벨 인코딩
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train_full)

# D. train/validation 분할 (random_state=42 조건 유지)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)
print(f"train 개수: {x_train.shape[0]}, val 개수: {x_val.shape[0]}")


# ============================================================
# 3. 데이터 증강 및 전처리 파이프라인
# ============================================================
print("\n2. 데이터 증강 설정 중...")

# 전이 학습 모델에 최적화된 ImageDataGenerator 설정
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15, # 이동 폭 증가
    height_shift_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    # 💡 MobileNetV2용 전처리 함수를 증강 파이프라인에 통합
    preprocessing_function=preprocess_input
)

# 검증 및 테스트 데이터에도 전처리 적용
x_val_processed = preprocess_input(x_val)
x_test_processed = preprocess_input(x_test)

train_generator = datagen.flow(x_train, y_train, batch_size=16)


# ============================================================
# 4. 전이 학습 모델 (MobileNetV2) 정의
# ============================================================
print("\n3. MobileNetV2 전이 학습 모델 설계 중...")





def build_transfer_model(input_shape):
    # MobileNetV2 로드 (사전 학습 가중치 'imagenet' 사용)
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False, # 상단 분류기 제외
        weights='imagenet'
    )

    # 1단계: 특징 추출 레이어 동결 (과적합 방지)
    base_model.trainable = False

    # 새로운 분류기(Head) 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x) # 특징 맵을 1차원으로 압축
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x) # 강력한 Dropout
    outputs = Dense(1, activation='sigmoid')(x) #이진 분류

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

model = build_transfer_model((IMG_SIZE, IMG_SIZE, 3))

# 1단계 컴파일: 분류기만 학습
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

model.summary()


# ============================================================
# 5. 모델 학습 (2단계 학습 전략)
# ============================================================
print("\n4. 모델 학습 시작 (2단계 전략 적용)...")

# 콜백 설정
es = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)

# --- 1단계 학습: Head 학습 (Base Model 동결) ---
print("--- 1단계: Head 학습 (분류기만 학습) ---")
model.fit(
    train_generator,
    steps_per_epoch=len(x_train) // 16,
    epochs=20, # 짧게 훈련
    validation_data=(x_val_processed, y_val),
    callbacks=[es, rlr],
    verbose=1
)

# --- 2단계 학습: Fine-tuning (미세 조정) ---
print("\n--- 2단계: Fine-tuning 시작 (Base Model 동결 해제) ---")
# Base Model의 가중치를 해제하고 아주 낮은 학습률로 전체 모델 학습
model.trainable = True

# 매우 낮은 학습률 재설정
model.compile(
    optimizer=Adam(learning_rate=1e-5), # 💡 학습률을 100배 낮춰서 미세 조정
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

es_ft = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)

model.fit(
    train_generator,
    steps_per_epoch=len(x_train) // 16,
    epochs=50, # 추가 훈련
    validation_data=(x_val_processed, y_val),
    callbacks=[es_ft, rlr],
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val_processed, y_val, verbose=0)
print(f"\n최종 검증 정확도 (Fine-tuning 후): {val_acc:.5f}")


# ============================================================
# 6. 예측 + 제출 파일 생성
# ============================================================
print("\n5. 제출 파일 생성 중...")

# 테스트 데이터에 대한 예측 확률
y_pred_proba = model.predict(x_test_processed)

# 0/1 변환
y_pred = np.round(y_pred_proba).astype(int).flatten()

# 문자열 라벨로 변환
y_pred_labels = label_encoder.inverse_transform(y_pred)

# submission.csv에 기록
df_submission[result_col] = y_pred_labels

submission_filename = "my_submission_TransferLearning_XAI_04.csv"
df_submission.to_csv(submission_filename, index=False)

print(f"\n✅ 제출 파일 저장 완료: {submission_filename}")
print(f"✨ Baseline({0.60784})보다 높은 정확도(2.0점) 달성을 기대합니다.")

1. 데이터 로드 및 전처리 중...
train 개수: 161, val 개수: 41

2. 데이터 증강 설정 중...

3. MobileNetV2 전이 학습 모델 설계 중...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

...


4. 모델 학습 시작 (2단계 전략 적용)...
--- 1단계: Head 학습 (분류기만 학습) ---
Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 4s/step - accuracy: 0.6691 - loss: 0.6791 - val_accuracy: 0.7073 - val_loss: 0.5339 - learning_rate: 0.0010
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.1724 - val_accuracy: 0.7317 - val_loss: 0.5452 - learning_rate: 0.0010
Epoch 3/20
 2/10 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.8824 - loss: 0.3799

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.7799 - loss: 0.4342 - val_accuracy: 0.7805 - val_loss: 0.4268 - learning_rate: 0.0010
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7500 - loss: 0.4616 - val_accuracy: 0.7805 - val_loss: 0.4226 - learning_rate: 0.0010
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.8879 - loss: 0.2756 - val_accuracy: 0.7561 - val_loss: 0.4102 - learning_rate: 0.0010
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7500 - loss: 0.5113 - val_accuracy: 0.7805 - val_loss: 0.4003 - learning_rate: 0.0010
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.8765 - loss: 0.2844 - val_accuracy: 0.8049 - val_loss: 0.3911 - learning_rate: 0.0010
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9375 - loss: 0.1596 - val_accuracy: 0.8049 - val_loss: 0.3812 - learning_rate: 0.0010
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.8855 - loss: 0.2387 - val_accuracy: 0.8